# Day8_3: 시계열 정상성과 진단 (ADF, KPSS, ACF, PACF)

## 📚 학습 목표

**Part 1: 기초**
1. 정상성(Stationarity)의 개념과 중요성 이해하기
2. 비정상 시계열의 특징 파악하기
3. ADF 검정으로 정상성 통계적 진단하기
4. KPSS 검정으로 정상성 교차 검증하기
5. 두 검정 결과를 종합하여 판단하기

**Part 2: 심화**
1. 로그 변환으로 분산 안정화하기
2. 차분(Differencing)으로 추세 제거하기
3. ACF(자기상관함수) 플롯 해석하기
4. PACF(편자기상관함수) 플롯 해석하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 정상성 검정 | 예측 모델 적용 전 필수 진단 | ARIMA 모델링 전 데이터 검증 |
| ADF/KPSS | 객관적 정상성 판단 | 주가 데이터 분석 가능 여부 결정 |
| 로그 변환 | 변동성 안정화 | 급등하는 주가/매출 데이터 처리 |
| 차분 | 추세 제거 | 상승 트렌드 제거 후 패턴 분석 |
| ACF/PACF | ARIMA 모델 파라미터 결정 | p, q 값 추정 |

**분석가 관점**: 시계열 예측 모델(ARIMA, SARIMA 등)은 데이터가 정상성을 만족해야 합니다. 정상성 검정과 변환은 예측 모델링의 필수 첫 단계입니다!

---

In [6]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 통계 검정 라이브러리
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 로드 완료")

ModuleNotFoundError: No module named 'matplotlib'

# Part 1: 기초

---

## 1.1 정상성(Stationarity)이란?

### 정상성의 정의

**정상성(Stationarity)**이란 시계열 데이터의 통계적 특성(평균, 분산 등)이 시간의 흐름에 따라 변하지 않는 것을 의미합니다.

마치 **잔잔한 호수**처럼, 데이터의 전반적인 모습이 어느 시점에서 보아도 일정하게 유지되는 상태입니다.

### 정상 시계열의 조건

1. **일정한 평균**: 시간이 지나도 평균값이 변하지 않음
2. **일정한 분산**: 시간이 지나도 변동 폭이 일정함
3. **일정한 공분산**: 두 시점 간의 공분산이 시차에만 의존함

### 왜 정상성이 중요한가?

- 대부분의 시계열 예측 모델(ARIMA 등)은 **데이터가 정상성을 만족**한다고 가정
- 정상성이 있어야 과거 패턴을 기반으로 미래를 예측 가능
- **비정상 데이터**는 모델링 전에 반드시 **정상화 변환** 필요

In [ ]:
# 정상 vs 비정상 시계열 비교
np.random.seed(42)

# 1. 정상 시계열 (백색 소음)
stationary_data = np.random.randn(200)

# 2. 비정상 시계열 (랜덤 워크 + 추세)
trend = np.arange(200) * 0.1
random_walk = np.cumsum(np.random.randn(200))
non_stationary_data = random_walk + trend

# DataFrame 생성
dates = pd.date_range('2024-01-01', periods=200, freq='D')
df_compare = pd.DataFrame({
    'stationary': stationary_data,
    'non_stationary': non_stationary_data
}, index=dates)

print("정상 시계열 통계:")
print(f"  평균: {stationary_data.mean():.2f}")
print(f"  표준편차: {stationary_data.std():.2f}")
print("\n비정상 시계열 통계:")
print(f"  평균: {non_stationary_data.mean():.2f}")
print(f"  표준편차: {non_stationary_data.std():.2f}")

In [ ]:
# 시각화: 정상 vs 비정상
fig = make_subplots(rows=1, cols=2, subplot_titles=('✅ 정상 시계열 (백색 소음)', '❌ 비정상 시계열 (추세 + 랜덤워크)'))

fig.add_trace(
    go.Scatter(x=dates, y=df_compare['stationary'], mode='lines', name='정상', line=dict(color='green')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=dates, y=df_compare['non_stationary'], mode='lines', name='비정상', line=dict(color='red')),
    row=1, col=2
)

fig.update_layout(height=400, title_text='정상 vs 비정상 시계열 비교', showlegend=False)
fig.show()

### 💡 실무 예시: 주가 데이터

주가는 대표적인 비정상 시계열입니다:
- **상승 추세**: 장기적으로 우상향하는 경향
- **변동성 변화**: 시간에 따라 변동 폭이 달라짐
- 이런 데이터는 ARIMA 모델을 바로 적용할 수 없음

---

## 1.2 비정상 시계열의 특징

### 비정상성의 주요 원인

| 유형 | 특징 | 예시 |
|------|------|------|
| **추세(Trend)** | 시간에 따라 평균이 증가/감소 | 경제 성장, 인구 증가 |
| **계절성(Seasonality)** | 일정 주기로 반복되는 패턴 | 여름 아이스크림 판매, 연말 쇼핑 |
| **분산 변화** | 시간에 따라 변동 폭이 달라짐 | 금융 위기 시 주가 변동성 증가 |
| **구조적 변화** | 특정 시점에서 패턴이 급변 | COVID-19 이후 경제 지표 |

In [ ]:
# 다양한 비정상성 유형 시각화
np.random.seed(123)
n = 200

# 1. 추세 (Trend)
trend_data = np.linspace(0, 50, n) + np.random.randn(n) * 3

# 2. 계절성 (Seasonality)
seasonal_data = 20 * np.sin(np.linspace(0, 8*np.pi, n)) + np.random.randn(n) * 2

# 3. 분산 변화 (Heteroscedasticity)
variance_data = np.random.randn(n) * np.linspace(1, 10, n)

# 4. 추세 + 계절성
combined_data = np.linspace(0, 30, n) + 15 * np.sin(np.linspace(0, 8*np.pi, n)) + np.random.randn(n) * 3

fig = make_subplots(rows=2, cols=2, subplot_titles=(
    '1. 추세 (Trend)', 
    '2. 계절성 (Seasonality)',
    '3. 분산 변화 (Heteroscedasticity)',
    '4. 추세 + 계절성'
))

fig.add_trace(go.Scatter(y=trend_data, mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(y=seasonal_data, mode='lines', line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(y=variance_data, mode='lines', line=dict(color='orange')), row=2, col=1)
fig.add_trace(go.Scatter(y=combined_data, mode='lines', line=dict(color='red')), row=2, col=2)

fig.update_layout(height=600, title_text='비정상 시계열의 유형', showlegend=False)
fig.show()

---

## 1.3 ADF 검정 (Augmented Dickey-Fuller Test)

### ADF 검정이란?

**ADF 검정**은 시계열 데이터의 정상성을 통계적으로 확인하는 가장 대표적인 방법입니다.

**가설 설정**:
- **귀무가설 (H0)**: 데이터에 **단위근(Unit Root)**이 존재한다 → **비정상 시계열**
- **대립가설 (H1)**: 단위근이 없다 → **정상 시계열**

**해석**:
- **p-value < 0.05**: 귀무가설 기각 → **정상 시계열**
- **p-value ≥ 0.05**: 귀무가설 채택 → **비정상 시계열**

In [ ]:
# ADF 검정 함수 정의
def adf_test(series, name=''):
    """
    ADF 검정을 수행하고 결과를 출력합니다.
    
    Parameters:
    -----------
    series : pd.Series
        검정할 시계열 데이터
    name : str
        출력 시 표시할 이름
    """
    result = adfuller(series.dropna())
    
    print(f'>>> ADF 검정: {name} <<<')
    print(f'- ADF 통계량: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    print(f'- 사용된 시차: {result[2]}')
    print(f'- 관측치 수: {result[3]}')
    print('- 임계값:')
    for key, value in result[4].items():
        print(f'    {key}: {value:.4f}')
    
    if result[1] < 0.05:
        print("\n✅ 결론: p-value < 0.05 → 정상 시계열 (귀무가설 기각)")
        return True
    else:
        print("\n❌ 결론: p-value ≥ 0.05 → 비정상 시계열 (귀무가설 채택)")
        return False

In [ ]:
# 정상 시계열 검정
print("=" * 50)
adf_test(pd.Series(stationary_data), '정상 시계열')
print("\n" + "=" * 50)
# 비정상 시계열 검정
adf_test(pd.Series(non_stationary_data), '비정상 시계열')

### 💡 실무 예시: 실제 경제 데이터로 ADF 검정

yfinance를 사용하여 실제 주가 데이터의 정상성을 검정해봅니다.

In [ ]:
# yfinance로 실제 주가 데이터 가져오기
import yfinance as yf

# 애플 주가 (2년치)
ticker = yf.Ticker('AAPL')
stock_data = ticker.history(period='2y')
stock_price = stock_data['Close']

print(f"애플 주가 데이터: {len(stock_price)}일")
print(stock_price.tail())

In [ ]:
# 주가 시각화
fig = px.line(stock_price, title='애플(AAPL) 종가 추이')
fig.update_layout(xaxis_title='날짜', yaxis_title='종가 ($)')
fig.show()

In [ ]:
# 주가 원본에 대한 ADF 검정
adf_test(stock_price, '애플 주가 (원본)')

---

## 1.4 KPSS 검정 (Kwiatkowski-Phillips-Schmidt-Shin Test)

### KPSS 검정이란?

**KPSS 검정**은 ADF와 **귀무가설이 반대**인 검정 방법입니다.

**가설 설정**:
- **귀무가설 (H0)**: 데이터가 **(추세에 대해) 정상적**이다
- **대립가설 (H1)**: 데이터가 **비정상적**이다

**해석**:
- **p-value > 0.05**: 귀무가설 채택 → **정상 시계열**
- **p-value ≤ 0.05**: 귀무가설 기각 → **비정상 시계열**

⚠️ **주의**: ADF와 KPSS는 귀무가설이 반대이므로 해석에 주의!

In [7]:
# KPSS 검정 함수 정의
def kpss_test(series, name=''):
    """
    KPSS 검정을 수행하고 결과를 출력합니다.
    
    Parameters:
    -----------
    series : pd.Series
        검정할 시계열 데이터
    name : str
        출력 시 표시할 이름
    """
    # 'c'는 상수(level)에 대한 정상성 검정, 'ct'는 추세(trend)에 대한 검정
    result = kpss(series.dropna(), regression='c', nlags='auto')
    
    print(f'>>> KPSS 검정: {name} <<<')
    print(f'- KPSS 통계량: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    print(f'- 사용된 시차: {result[2]}')
    print('- 임계값:')
    for key, value in result[3].items():
        print(f'    {key}: {value:.4f}')
    
    if result[1] > 0.05:
        print("\n✅ 결론: p-value > 0.05 → 정상 시계열 (귀무가설 채택)")
        return True
    else:
        print("\n❌ 결론: p-value ≤ 0.05 → 비정상 시계열 (귀무가설 기각)")
        return False

In [8]:
# 정상 시계열 KPSS 검정
print("=" * 50)
kpss_test(pd.Series(stationary_data), '정상 시계열')
print("\n" + "=" * 50)
# 비정상 시계열 KPSS 검정
kpss_test(pd.Series(non_stationary_data), '비정상 시계열')

>>> KPSS 검정: 정상 시계열 <<<
- KPSS 통계량: 0.1797
- p-value: 0.1000
- 사용된 시차: 2
- 임계값:
    10%: 0.3470
    5%: 0.4630
    2.5%: 0.5740
    1%: 0.7390

✅ 결론: p-value > 0.05 → 정상 시계열 (귀무가설 채택)

>>> KPSS 검정: 비정상 시계열 <<<
- KPSS 통계량: 1.9812
- p-value: 0.0100
- 사용된 시차: 9
- 임계값:
    10%: 0.3470
    5%: 0.4630
    2.5%: 0.5740
    1%: 0.7390

❌ 결론: p-value ≤ 0.05 → 비정상 시계열 (귀무가설 기각)


/tmp/ipykernel_19698/2792140979.py:14: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.


/tmp/ipykernel_19698/2792140979.py:14: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




False

---

## 1.5 ADF + KPSS 종합 판단

### 두 검정 결과 해석

| ADF (p<0.05) | KPSS (p>0.05) | 결론 |
|--------------|---------------|------|
| ✅ 정상 | ✅ 정상 | **정상 시계열** (확실) |
| ❌ 비정상 | ❌ 비정상 | **비정상 시계열** (확실) |
| ✅ 정상 | ❌ 비정상 | 차분 정상성 (Difference Stationary) |
| ❌ 비정상 | ✅ 정상 | 추세 정상성 (Trend Stationary) |

In [9]:
# 종합 정상성 검정 함수
def check_stationarity(series, name=''):
    """
    ADF와 KPSS 검정을 종합하여 정상성을 판단합니다.
    """
    print(f"\n{'='*60}")
    print(f"📊 종합 정상성 검정: {name}")
    print(f"{'='*60}\n")
    
    # ADF 검정
    adf_result = adfuller(series.dropna())
    adf_stationary = adf_result[1] < 0.05
    
    # KPSS 검정
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    kpss_stationary = kpss_result[1] > 0.05
    
    print(f"ADF 검정:  p-value = {adf_result[1]:.4f} → {'✅ 정상' if adf_stationary else '❌ 비정상'}")  #⭐ 좀 특이한 if문 기억해놓기!⭐
    print(f"KPSS 검정: p-value = {kpss_result[1]:.4f} → {'✅ 정상' if kpss_stationary else '❌ 비정상'}")
    print()
    
    if adf_stationary and kpss_stationary:
        print("🎯 종합 결론: 정상 시계열 (두 검정 모두 정상)")
        return 'stationary'
    elif not adf_stationary and not kpss_stationary:
        print("🎯 종합 결론: 비정상 시계열 (두 검정 모두 비정상)")
        return 'non-stationary'
    elif adf_stationary and not kpss_stationary:
        print("🎯 종합 결론: 차분 정상성 (Difference Stationary)")
        return 'difference-stationary'
    else:
        print("🎯 종합 결론: 추세 정상성 (Trend Stationary)")
        return 'trend-stationary'

In [10]:
# 종합 검정 수행
check_stationarity(pd.Series(stationary_data), '백색 소음')
check_stationarity(pd.Series(non_stationary_data), '랜덤워크+추세')
check_stationarity(stock_price, '애플 주가')


📊 종합 정상성 검정: 백색 소음

ADF 검정:  p-value = 0.0000 → ✅ 정상
KPSS 검정: p-value = 0.1000 → ✅ 정상

🎯 종합 결론: 정상 시계열 (두 검정 모두 정상)

📊 종합 정상성 검정: 랜덤워크+추세

ADF 검정:  p-value = 0.8024 → ❌ 비정상
KPSS 검정: p-value = 0.0100 → ❌ 비정상

🎯 종합 결론: 비정상 시계열 (두 검정 모두 비정상)


/tmp/ipykernel_19698/3798094667.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.


/tmp/ipykernel_19698/3798094667.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




NameError: name 'stock_price' is not defined

---

# Part 2: 심화

---

## 2.1 로그 변환 (Log Transformation)

### 로그 변환의 목적

- **분산 안정화**: 시간에 따라 변동성이 커지는 데이터에 적용
- 주가, GDP 등 **지수적으로 성장**하는 데이터에 효과적
- 변환 후 데이터의 변동 폭이 일정해짐

### 수식

$$Y'_t = \log(Y_t)$$

**장점**:
- 큰 값의 영향을 줄임
- 분산을 안정화시킴
- 로그 수익률 계산에 유용

In [11]:
# 로그 변환 예시
# 주가 원본
stock_original = stock_price.copy()

# 로그 변환
stock_log = np.log(stock_price)

print("원본 주가 통계:")
print(f"  평균: ${stock_original.mean():.2f}")
print(f"  표준편차: ${stock_original.std():.2f}")
print(f"  변동계수(CV): {stock_original.std() / stock_original.mean() * 100:.2f}%")

print("\n로그 변환 후:")
print(f"  평균: {stock_log.mean():.4f}")
print(f"  표준편차: {stock_log.std():.4f}")
print(f"  변동계수(CV): {stock_log.std() / stock_log.mean() * 100:.2f}%")

NameError: name 'stock_price' is not defined

In [12]:
# 시각화: 원본 vs 로그 변환
fig = make_subplots(rows=1, cols=2, subplot_titles=('원본 주가', '로그 변환 후'))

fig.add_trace(
    go.Scatter(x=stock_original.index, y=stock_original, mode='lines', name='원본', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=stock_log.index, y=stock_log, mode='lines', name='로그', line=dict(color='green')),
    row=1, col=2
)

fig.update_layout(height=400, title_text='로그 변환: 분산 안정화', showlegend=False)
fig.show()

NameError: name 'stock_original' is not defined

In [ ]:
# 로그 변환 후 정상성 검정
check_stationarity(stock_log, '애플 주가 (로그 변환)')

### 💡 실무 팁: 로그 변환만으로는 추세가 제거되지 않음

로그 변환은 **분산만 안정화**시킬 뿐, **추세는 여전히 존재**합니다.
추세를 제거하려면 **차분**이 필요합니다.

---

## 2.2 차분 (Differencing)

### 차분의 목적

- **추세 제거**: 시간에 따라 평균이 변하는 것을 제거
- 비정상 시계열을 정상 시계열로 변환

### 1차 차분 (First Differencing)

$$Y'_t = Y_t - Y_{t-1}$$

### 2차 차분 (Second Differencing)

$$Y''_t = Y'_t - Y'_{t-1} = (Y_t - Y_{t-1}) - (Y_{t-1} - Y_{t-2})$$

**pandas 메서드**: `.diff()`

In [ ]:
# 1차 차분 적용
stock_diff1 = stock_price.diff().dropna()

print("1차 차분 결과 (처음 10개):")
print(stock_diff1.head(10))
print(f"\n차분 후 데이터 개수: {len(stock_diff1)} (원본: {len(stock_price)})")

In [ ]:
# 시각화: 원본 vs 1차 차분
fig = make_subplots(rows=2, cols=1, subplot_titles=('원본 주가', '1차 차분 (일일 변동)'))

fig.add_trace(
    go.Scatter(x=stock_price.index, y=stock_price, mode='lines', name='원본', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=stock_diff1.index, y=stock_diff1, mode='lines', name='1차 차분', line=dict(color='red')),
    row=2, col=1
)

fig.update_layout(height=500, title_text='차분: 추세 제거', showlegend=False)
fig.show()

In [ ]:
# 1차 차분 후 정상성 검정
check_stationarity(stock_diff1, '애플 주가 (1차 차분)')

### 로그 변환 + 차분 (권장 순서)

일반적으로 **로그 변환을 먼저** 적용한 후 **차분**을 수행합니다.

이유:
1. 로그 변환으로 분산 안정화
2. 차분으로 추세 제거
3. 로그 차분 = 로그 수익률 (금융에서 많이 사용)⭐

In [13]:
# 로그 변환 후 1차 차분 (로그 수익률)
stock_log_diff = np.log(stock_price).diff().dropna()

print("로그 수익률 (처음 10개):")
print(stock_log_diff.head(10))
print(f"\n평균 로그 수익률: {stock_log_diff.mean():.6f}")
print(f"표준편차: {stock_log_diff.std():.6f}")

NameError: name 'stock_price' is not defined

In [ ]:
# 시각화: 변환 단계별 비교
fig = make_subplots(rows=2, cols=2, subplot_titles=(
    '1. 원본',
    '2. 로그 변환',
    '3. 1차 차분',
    '4. 로그 변환 + 차분 (로그 수익률)'
))

fig.add_trace(go.Scatter(x=stock_price.index, y=stock_price, mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=stock_log.index, y=stock_log, mode='lines', line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(x=stock_diff1.index, y=stock_diff1, mode='lines', line=dict(color='orange')), row=2, col=1)
fig.add_trace(go.Scatter(x=stock_log_diff.index, y=stock_log_diff, mode='lines', line=dict(color='red')), row=2, col=2)

fig.update_layout(height=600, title_text='데이터 변환 단계별 비교', showlegend=False)
fig.show()

In [14]:
# 로그 변환 + 차분 후 정상성 검정
check_stationarity(stock_log_diff, '애플 주가 (로그+차분)')

NameError: name 'stock_log_diff' is not defined

---

## 2.3 ACF (자기상관함수) 플롯

### ACF란?

**ACF (Autocorrelation Function, 자기상관함수)**는 시계열 데이터에서 현재 값과 과거 값 사이의 상관관계를 측정합니다.

- **lag k**에서의 ACF = $Y_t$와 $Y_{t-k}$ 간의 상관계수
- **정상 시계열**: ACF가 빠르게 0으로 수렴
- **비정상 시계열**: ACF가 천천히 감소

### ACF 활용

1. **정상성 진단**: ACF가 빠르게 0으로 수렴하면 정상
2. **MA(q) 차수 결정**: ACF가 lag q 이후 급격히 0으로 절단 → MA(q)

In [ ]:
# ACF 플롯 - 비정상 시계열 (원본 주가)
fig, ax = plt.subplots(figsize=(12, 4))
plot_acf(stock_price.dropna(), lags=40, ax=ax)
ax.set_title('ACF: 원본 주가 (비정상 시계열)')
plt.tight_layout()
plt.show()

print("▶ 해석: ACF가 천천히 감소 → 비정상 시계열")

###⭐한글 깨지는 거 해결해주는 라이브러리 : https://github.com/seongminp/koreanize-matplotlib

NameError: name 'plt' is not defined

In [ ]:
# ACF 플롯 - 정상 시계열 (로그+차분 후)
fig, ax = plt.subplots(figsize=(12, 4))
plot_acf(stock_log_diff.dropna(), lags=40, ax=ax)
ax.set_title('ACF: 로그 수익률 (정상 시계열)')
plt.tight_layout()
plt.show()

print("▶ 해석: ACF가 빠르게 0으로 수렴 → 정상 시계열")

#렉(지연)이 아주 적음

---

## 2.4 PACF (편자기상관함수) 플롯

### PACF란?

**PACF (Partial Autocorrelation Function, 편자기상관함수)**는 중간 시점들의 영향을 제외하고, 오직 두 시점 $(Y_t, Y_{t-k})$의 순수한 상관관계를 측정합니다.

### ACF vs PACF

| 구분 | ACF | PACF |
|------|-----|------|
| 측정 대상 | $Y_t$와 $Y_{t-k}$의 전체 상관 | 중간 시점 제외한 순수 상관 |
| 용도 | MA(q) 차수 결정 | AR(p) 차수 결정 |
| 절단 패턴 | lag q 이후 절단 → MA(q) | lag p 이후 절단 → AR(p) |

### ARIMA 파라미터 결정

| 모델 | ACF 플롯 특징 | PACF 플롯 특징 |
|------|-------------|---------------|
| **AR(p)** | 점차 감소 | lag p 이후 급격히 절단 |
| **MA(q)** | lag q 이후 급격히 절단 | 점차 감소 |
| **ARMA** | 점차 감소 | 점차 감소 |

In [ ]:
# PACF 플롯 - 정상 시계열 (로그+차분 후)
fig, ax = plt.subplots(figsize=(12, 4))
plot_pacf(stock_log_diff.dropna(), lags=40, ax=ax, method='ywm')
ax.set_title('PACF: 로그 수익률')
plt.tight_layout()
plt.show()

print("▶ 해석: PACF 패턴을 보고 AR(p)의 p 값을 추정")

In [ ]:
# ACF + PACF 함께 보기
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(stock_log_diff.dropna(), lags=30, ax=axes[0])
axes[0].set_title('ACF: 로그 수익률')

plot_pacf(stock_log_diff.dropna(), lags=30, ax=axes[1], method='ywm')
axes[1].set_title('PACF: 로그 수익률')

plt.tight_layout()
plt.show()

print("▶ ARIMA 파라미터 추정:")
print("   - ACF가 빠르게 절단되면 MA(q) 모델 고려")
print("   - PACF가 빠르게 절단되면 AR(p) 모델 고려")

### 💡 실무 예시: 정상화 파이프라인

실무에서는 다음 순서로 정상화를 진행합니다:

1. **시각화**: 원본 데이터 플롯으로 추세/계절성 확인
2. **검정**: ADF + KPSS로 정상성 진단
3. **로그 변환**: 분산이 변하면 적용
4. **차분**: 추세가 있으면 적용
5. **재검정**: 변환 후 정상성 재확인
6. **ACF/PACF**: ARIMA 파라미터 추정

In [ ]:
# 정상화 파이프라인 함수
def stationarize_pipeline(series, name=''):
    """
    시계열 데이터 정상화 파이프라인
    """
    print(f"\n{'='*60}")
    print(f"📊 정상화 파이프라인: {name}")
    print(f"{'='*60}")
    
    results = {'original': series}
    
    # 1단계: 원본 검정
    print("\n[1단계] 원본 데이터 검정")
    status = check_stationarity(series, '원본')
    
    if status == 'stationary':
        print("\n✅ 이미 정상 시계열입니다!")
        return series, results
    
    # 2단계: 로그 변환
    print("\n[2단계] 로그 변환")  #정상성이 없으면 로그변환 하겠어~!
    series_log = np.log(series)
    results['log'] = series_log
    status = check_stationarity(series_log, '로그 변환')
    
    if status == 'stationary':
        return series_log, results
    
    # 3단계: 1차 차분
    print("\n[3단계] 로그 + 1차 차분")
    series_log_diff = series_log.diff().dropna()
    results['log_diff'] = series_log_diff
    status = check_stationarity(series_log_diff, '로그+1차차분')
    
    if status == 'stationary':
        return series_log_diff, results
    
    # 4단계: 2차 차분
    print("\n[4단계] 로그 + 2차 차분")
    series_log_diff2 = series_log.diff().diff().dropna()
    results['log_diff2'] = series_log_diff2
    status = check_stationarity(series_log_diff2, '로그+2차차분')
    
    return series_log_diff2, results

In [15]:
# 파이프라인 실행
stationary_series, transform_results = stationarize_pipeline(stock_price, '애플 주가')

NameError: name 'stationarize_pipeline' is not defined

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 정상 vs 비정상 구분하기 ⭐

**문제**: 다음 두 시계열 중 어느 것이 정상 시계열인지 시각화하여 판단하세요.

```python
np.random.seed(100)
series_a = pd.Series(np.random.randn(100))
series_b = pd.Series(np.cumsum(np.random.randn(100)))
```

**기대 결과**: 어떤 시리즈가 정상인지 답변

In [ ]:
# 여기에 코드를 작성하세요


### Q2. ADF 검정 수행하기 ⭐⭐

**문제**: 다음 시계열 데이터에 대해 ADF 검정을 수행하고 정상성을 판단하세요.

```python
np.random.seed(42)
test_series = pd.Series(np.linspace(0, 10, 100) + np.random.randn(100) * 2)
```

**기대 결과**: ADF 통계량, p-value, 정상성 판단

In [ ]:
# 여기에 코드를 작성하세요


### Q3. KPSS 검정 수행하기 ⭐⭐

**문제**: Q2의 동일한 데이터에 대해 KPSS 검정을 수행하세요.

**기대 결과**: KPSS 통계량, p-value, 정상성 판단, ADF 결과와 비교

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 1차 차분으로 정상화하기 ⭐⭐⭐

**문제**: Q2의 데이터에 1차 차분을 적용한 후 ADF 검정으로 정상성을 확인하세요.

**기대 결과**: 차분 후 데이터의 ADF p-value < 0.05

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 로그 변환 적용하기 ⭐⭐⭐

**문제**: 다음 지수 성장 데이터에 로그 변환을 적용하고, 변환 전후의 분산을 비교하세요.

```python
np.random.seed(50)
exp_data = pd.Series(100 * np.exp(np.linspace(0, 2, 100)) + np.random.randn(100) * 10)
```

**기대 결과**: 변환 전후 분산 비교, 시각화

In [ ]:
# 여기에 코드를 작성하세요


### Q6. ACF 플롯 해석하기 ⭐⭐⭐

**문제**: 다음 두 시계열의 ACF 플롯을 그리고, 어느 것이 정상 시계열인지 ACF 패턴으로 판단하세요.

```python
np.random.seed(123)
stationary = pd.Series(np.random.randn(200))
non_stationary = pd.Series(np.cumsum(np.random.randn(200)))
```

**기대 결과**: 두 ACF 플롯 비교 및 해석

In [ ]:
# 여기에 코드를 작성하세요


### Q7. ADF + KPSS 종합 판단하기 ⭐⭐⭐⭐

**문제**: yfinance로 삼성전자(005930.KS) 주가를 가져와 ADF와 KPSS 검정을 모두 수행하고, 종합 결론을 내리세요.

**기대 결과**: 두 검정 결과와 종합 판단

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 로그 변환 + 차분 파이프라인 ⭐⭐⭐⭐

**문제**: Q7의 삼성전자 주가에 로그 변환 후 1차 차분을 적용하고, 각 단계별 정상성을 확인하세요.

**기대 결과**: 원본 → 로그 → 로그차분 각 단계별 ADF 결과

In [ ]:
# 여기에 코드를 작성하세요


### Q9. ACF/PACF로 모델 파라미터 추정 ⭐⭐⭐⭐⭐

**문제**: Q8에서 정상화된 데이터의 ACF/PACF 플롯을 그리고, ARIMA 모델의 p, q 값을 추정하세요.

**힌트**:
- PACF가 lag p 이후 절단 → AR(p)
- ACF가 lag q 이후 절단 → MA(q)

**기대 결과**: ACF/PACF 플롯과 p, q 추정값

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 완전한 정상화 파이프라인 구축 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 만족하는 정상화 파이프라인 함수를 작성하세요.

**요구사항**:
1. 입력: 시계열 데이터 (pd.Series)
2. 단계별로 ADF 검정 수행 (원본 → 로그 → 1차차분 → 2차차분)
3. 정상성 만족하는 즉시 해당 변환 데이터 반환
4. 각 단계별 시각화 (Plotly 서브플롯)
5. 최종 정상화된 데이터의 ACF/PACF 플롯 출력

**테스트 데이터**:
```python
np.random.seed(777)
test_data = pd.Series(100 * np.exp(np.linspace(0, 1, 200)) + np.random.randn(200) * 5)
```

**기대 결과**: 정상화된 시계열과 ACF/PACF 플롯

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| **정상성** | 평균/분산이 시간에 따라 일정 | ARIMA 모델 적용 전 필수 조건 |
| **ADF 검정** | H0: 비정상, p<0.05면 정상 | 정상성 통계적 진단 |
| **KPSS 검정** | H0: 정상, p>0.05면 정상 | ADF와 교차 검증 |
| **종합 판단** | ADF+KPSS 결과 조합 | 확실한 정상성 판단 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 목적 |
|------|------|------|
| **로그 변환** | `np.log(series)` | 분산 안정화 |
| **1차 차분** | `series.diff()` | 추세 제거 |
| **2차 차분** | `series.diff().diff()` | 강한 추세 제거 |
| **ACF 플롯** | `plot_acf(series, lags=40)` | MA(q) 파라미터 추정 |
| **PACF 플롯** | `plot_pacf(series, lags=40)` | AR(p) 파라미터 추정 |

### 💡 실무 팁

1. **변환 순서**: 로그 변환 → 차분 (분산 안정화 먼저)
2. **차분 횟수**: 보통 1~2차 차분이면 충분, 3차 이상은 과적합 위험
3. **교차 검증**: ADF와 KPSS 결과가 다르면 보수적으로 판단 (비정상으로)
4. **로그 수익률**: 금융 데이터에서 `np.log(series).diff()`가 표준
5. **ACF 해석**: 천천히 감소하면 비정상, 빠르게 0으로 수렴하면 정상
6. **ARIMA (p,d,q)**: d는 차분 횟수, p는 PACF 절단점, q는 ACF 절단점
7. **계절성**: 계절 데이터는 계절 차분도 고려 (SARIMA)